### Whoop Data Importer

In [ ]:
import pandas as pd
import requests 
import json
from datetime import datetime  # datetime parsing
import pytz  # timezone adjusting
import csv  # for making csv files
import os
import seaborn as sns
import numpy as np
import getpass


import plotly.express as px
import dash
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'


# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use('ggplot') # optional: for ggplot-like style

# check for latest version of Matplotlib
print ('Matplotlib version: ', mpl.__version__) # >= 2.0.0

In [ ]:
#################################################################
# Shift+Enter to enter password
username = input("username")
password = getpass.getpass("password")
#################################################################


# Post credentials
r = requests.post("https://api-7.whoop.com/oauth/token", json={
    "grant_type": "password",
    "issueRefresh": False,
    "password": password,
    "username": username
})

# Exit if fail


if r.status_code != 200:
    print("Fail - Credentials rejected.")
    exit()
else:
    print("Success - Credentials accepted")

# Set userid/token variables
userid = r.json()['user']['id']
access_token = r.json()['access_token']

In [ ]:
#################################################################
# GET DATA

# Download data
url = 'https://api-7.whoop.com/users/{}/cycles'.format(userid)

params = {
    'start': '2000-01-01T00:00:00.000Z',
    'end': '2030-01-01T00:00:00.000Z'
}

headers = {
    'Authorization': 'bearer {}'.format(access_token)
}

r = requests.get(url, params=params, headers=headers)


if r.status_code != 200:
    print("Fail - User ID / auth token rejected.")
    exit()
else:
    print("Success - User ID / auth token accepted")
    print("Whoop JSON imported successfully")

In [ ]:
#Displaying the raw Json
r.json()

In [ ]:
data_raw = r.json()

In [ ]:
for l in data_raw:
    for u in l["sleep"]['sleeps']:
        print(u)

In [ ]:
#################################################################
# PARSE/TRANSFORM DATA



def time_parse(time_string, offset_string):
    # Switch sign on offset
    offset_string = offset_string.replace(
        '-', '+') if offset_string.count('-') else offset_string.replace('+', '-')
    # Remove tz from time and add offset, get to 19 characters
    time_string = time_string[:-(len(time_string) - 19)] + offset_string
    # Parse and format
    oldformat = '%Y-%m-%dT%H:%M:%S%z'
    newformat = '%Y-%m-%d %H:%M:%S'
    return datetime.strptime(time_string, oldformat).astimezone(pytz.utc).strftime(newformat)
    

In [ ]:
######Strip data from JSON --> list with dict 


# Make data list
data_summary = []

# Iterate through data
for d in data_raw:

    # Make record object with default values
    record = {
        'Recovery Date': None,
        'rMSSD': None,
        'Recovery Score' : None,
        'Resting Heart Rate' : None,
        'Calories' : None
    }

    if d['recovery'] and 'heartRateVariabilityRmssd' in d['recovery'] and d['recovery'] and 'score' in d['recovery']:
        
        record['Recovery Date'] = time_parse(
            d['recovery']['timestamp'],
            d['sleep']['sleeps'][0]['timezoneOffset'])
        record['rMSSD'] = d['recovery']['heartRateVariabilityRmssd'] * 1000.0
        
        
        
        record['Recovery Score'] = d['recovery']['score']
        record['rMSSD'] = d['recovery']['heartRateVariabilityRmssd'] * 1000.0
        record['Resting Heart Rate'] = d['recovery']['restingHeartRate']
        record['Calories'] = d['strain']['kilojoules'] / 4.184

    
    
    


        data_summary.append(record)

In [ ]:
sleep_summary = []

for e in data_raw:
    
    sleep_record = {
        'Sleep Date' : None,
        'Time in bed' : None,
        'Sleep Pref. Score' : None,
        'Sleep Effic.' : None,
        'Sleep Pref. Score' : None,
        'Disturbances' : None,
        'Light Sleep time' : None,
        'Deep Sleep time' : None,
        'REM Sleep time' : None,
        
    }
    
    for f in e['sleep']['sleeps']:
        sleep_record['Sleep Date'] = f['during']['lower']
        sleep_record['Time in bed'] = f['inBedDuration'] / 3.6e+6
        sleep_record['Sleep Pref. Score'] = f['score']
        sleep_record['Sleep Effic.'] = f['sleepEfficiency'] *100
        sleep_record['Disturbances'] = f['disturbanceCount']
        sleep_record['Light Sleep time'] = f['lightSleepDuration'] / 3.6e+6
        sleep_record['Deep Sleep time'] = f['slowWaveSleepDuration'] / 3.6e+6
        sleep_record['REM Sleep time'] = f['remSleepDuration'] / 3.6e+6
        
        
        sleep_summary.append(sleep_record)

In [ ]:
data_summary

In [ ]:
sleep_summary

In [ ]:
df_sleep = pd.DataFrame(sleep_summary)

In [ ]:
df_rec = pd.DataFrame(data_summary)

In [ ]:
df_comb = pd.concat([df_rec, df_sleep], axis=1, join='inner')

In [ ]:
df_comb['Recovery Date'] = pd.to_datetime(df_comb['Recovery Date'])

In [ ]:
df_comb['Total Sleep Time'] = df_comb['Light Sleep time']+df_comb['Deep Sleep time']+df_comb['REM Sleep time']

In [ ]:
df_comb['Rem Sleep %'] = df_comb['REM Sleep time']/df_comb['Total Sleep Time']
df_comb['Deep Sleep %'] = df_comb['Deep Sleep time']/df_comb['Total Sleep Time']
df_comb['Light Sleep %'] = df_comb['Light Sleep time']/df_comb['Total Sleep Time']

In [ ]:
df_comb['Year'] = pd.DatetimeIndex(df_comb['Recovery Date']).year

In [ ]:
df_comb['Month'] = pd.DatetimeIndex(df_comb['Recovery Date']).month

In [ ]:
df_comb['Recov Month-Yr'] = pd.to_datetime(df_comb['Recovery Date']).dt.to_period('M')

In [ ]:
df_comb.info()


In [ ]:
###Saving the dataframe to CSV##
df_comb.to_csv('WhoopData19-22.csv', index=True)

In [ ]:
df_comb

In [ ]:
yr_avg = df_comb.groupby(['Year']).mean()
yr_avg.reset_index(inplace=True)
yr_avg

In [ ]:
df_whoop_summary = df_comb.groupby(['Year','Month']).mean()
df_whoop_summary

In [ ]:
df_sleep = df_whoop_summary[['REM Sleep time','Deep Sleep time']]

In [ ]:
df_sleep.plot(kind='area', stacked=False, figsize=(20, 10))

In [ ]:
df_sleep['REM Sleep time']

In [ ]:
df_sleep.reset_index(inplace=True)
df_sleep

In [ ]:
df_comb_19 = df_comb[df_comb.Year == 2019]

In [ ]:
df_comb_20 = df_comb[df_comb.Year == 2020]

In [ ]:
df_comb_21 = df_comb[df_comb.Year == 2021]

In [ ]:
HRV_dst_19 = df_comb_19['rMSSD'].value_counts(bins=10, sort=False)
print(HRV_dst_19)

In [ ]:
HRV_dst_19.plot(kind='bar', stacked=False, figsize=(20, 10))

In [ ]:
HRV_dst_20 = df_comb_20['rMSSD'].value_counts(bins=10, sort=False)
print(HRV_dst_20)

In [ ]:
HRV_dst_20.plot(kind='bar', stacked=False, figsize=(20, 10))

In [ ]:
HRV_dst_21 = df_comb_21['rMSSD'].value_counts(bins=10, sort=False)
print(HRV_dst_21)